In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 42.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=fb6e3c818bd28cde912ca839ba10681c922d216ec61325a1473d43ca54ac6bcb
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
import json
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
sc = SparkContext("local","Finalproject")
spark = SparkSession(sc)

In [ ]:

"""
Functions
"""
#1: Most genre have books
#use this function and do map and reduce(wordcount)
def get_AllGenres(book_metadata):
  """
  this function output every genre assigned to every book in the book_metadata goodreads_books.json.gz
  """
  genre_list=[]
  book_dict=json.loads(book_metadata)
  if len(book_dict['popular_shelves']) != 0:
    for genre in  book_dict['popular_shelves']:
    #it is a list of genres
      genre_list.append(genre['name'])
  return genre_list

#2 most language for every genre

# use only this function as it should return for every book a list of genres assiciated with the language of the book
def get_Lang_with_Genres(book_metadata):
    """
    this function output every language assigned to every genre in the book_metadata goodreads_books.json.gz
    """
    genre_list=[]
    lang_genre_list=[]
    book_dict=json.loads(book_metadata)
    book_lang=book_dict['language_code']
    if len(book_dict['popular_shelves']) != 0:
        for genre in  book_dict['popular_shelves']:
            #it is a list of genres
            genre_list.append(genre['name'])
    if len(book_lang)!=0:
        lang_genre_list=[book_lang+'***'+i for i in genre_list]

    return lang_genre_list

#3-Most number of words for one writer
# this process will be on two Functions
# the first function is the most important as it will just use the writer id
# we can transform the writer id onto a writer name if we have time
def get_authorID(book_metadata):
    """
    this function output every author assigned to every book in the book_metadata goodreads_books.json.gz
    """
    authorsID_list=[]
    book_dict=json.loads(book_metadata)
    authors_list=book_dict['authors']
    if len(authors_list) != 0:
        for author in  authors_list:
            #it is a list of genres
            authorsID_list.append(author['author_id'])
    return authorsID_list


#4-Most books have reviews
# this Process will be on two functions also
# the first function is the important but it gives the number of reviews for a book id
# later we can transform the book id onto a book name

def get_bookID_reviews(review):
    """
    this function output every genre assigned to every book , using the reviews file : goodread_reviews_dedup.json.gz
    """
    BookID_list=[]
    review_dict=json.loads(review)
    book_id=review_dict['book_id']
    if len(book_id) != 0:
        BookID_list.append(book_id)
    return BookID_list

# I found that i can get the reviews number from the book metadata:
# but also use the same map function as mentinoed in requirement number 6
def get_books_reviews(book_metadata):
    """
    this function output number of reviews for every book in the book_metadata goodreads_books.json.gz
    """
    reviews_list=[]
    book_dict=json.loads(book_metadata)
    book_name=book_dict['title_without_series']
    reviews_num=book_dict['text_reviews_count']
    if len(reviews_num) != 0:
            reviews_list.append(book_name+'*-_-*'+reviews_num)
    return reviews_list

#5-Most readed books
# first of all I am assuming that spark is reading a csv file line-by-line as a text
# i also wrote another code if spark read csv like json (indexing with the column name not buy a number)
# this requirement will take 3 processess
# the first process is the important one as it will return the scv id of the book with the number of reads for it
# the second process should transform the csv id into regular book id then the third will transfrom the book id onto b

def get_read_books(interaction):
    """
    this will get the readerd books only , it should be used with the interactions file: goodreads_interactions.csv
    """
    booksIDS_list=[]
    interactiondata=json.loads(interaction)
    # is read is the third element
    is_read=interactiondata['is_read']
    # book id is the second element
    book_id=interactiondata['book_id']
    if is_read ==True:
      booksIDS_list.append(book_id)
    return booksIDS_list


# 6-Most “want to read” books
# this will be on one process only
# i will get the count of the 'to-read' shelf in popular shelves for every book book_metadata
#this function should return
# remember this funciton will not use code for map:>>>>> words.map(lambda key:(key,1))
# change map to code:>>>>>>>>     words.map(lambda key:(key.split('*-_-*')[0],int(key.split('*-_-*')[1])))

def get_to_read_books(book_metadata):
  """
  this functoin gets the number of to-reads for every book in the book_metadata goodreads_books.json.gz
  """
  books_with_toReadNum_list=[]
  book_dict=json.loads(book_metadata)
  book_name=book_dict['title_without_series']
  if len(book_dict['popular_shelves']) != 0:
    for genre in  book_dict['popular_shelves']:
    #it is a list of genres
      if genre['name']== 'to-read':
        books_with_toReadNum_list.append(book_name+'*-_-*'+genre['count'])
  return books_with_toReadNum_list

#7-Most year have published books


def get_year(book_metadata):
    """
    this function output every year assigned to every book in the book_metadata goodreads_books.json.gz
    """
    year_list=[]
    book_dict=json.loads(book_metadata)
    year=book_dict['publication_year']
    if len(year) != 0:
        year_list.append(year)
    return year_list




#10-Most double genre for books(children&Fantasy, romance&thriller,etc.). 2 genres, 3 genres
#There are 2 function , the first to get a comination of every 2 genres for every books
# the second function is not important
from itertools import combinations
def get_2genre_combination(book_metadata):
    """
    this function output every 2genre associated for every book in the book_metadata goodreads_books.json.gz
    """
    genre_syn=[]
    genre_list=[]
    avoid_list=['to-read','owned','currently-reading','favorites','books-i-own','kindle','ebook','library','default','owned-books','to-buy','ebooks','wish-list','my-books','my-library','e-book','audiobook','books','i-own','audiobooks','audio','read-in-2016','favourites','e-books','read-in-2015','read-in-2017','own-it','maybe','read-in-2014','abandoned','did-not-finish'] # add others if found
    book_dict=json.loads(book_metadata)
    if len(book_dict['popular_shelves']) != 0:
        for genre in  book_dict['popular_shelves']:
            #it is a list of genres
            genre_list.append(genre['name'])
        genre_list_mod=[i for i in genre_list if i not in avoid_list]
        genre_combination=combinations(genre_list_mod,2)
        for combination in genre_combination:
            genre_syn.append('*'.join(list(sorted(combination))))

    return genre_syn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sc = spark.sparkContext
#pathr='/content/drive/MyDrive/BigData_FinalProject/datasets/FINALPROJECTDATA/Metadata/goodreads_books.json.gz'
author_data=sc.textFile('/content/drive/MyDrive/BigData_FinalProject/datasets/FINALPROJECTDATA/Metadata/goodreads_book_authors.json.gz')
books_data = sc.textFile('/content/drive/MyDrive/BigData_FinalProject/datasets/FINALPROJECTDATA/Metadata/goodreads_books.json.gz')

In [ ]:
books_data.take(10)

['{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

In [ ]:
#functions are
"""
get_AllGenres
get_Lang_with_Genres
get_authorID
get_bookID_reviews
get_books_reviews
get_read_books
get_AllGenres
get_year
"""
def get_authorID(book_metadata):
    """
    this function output every author assigned to every book in the book_metadata goodreads_books.json.gz
    """
    authorsID_list=[]
    book_dict=json.loads(book_metadata)
    authors_list=book_dict['authors']
    if len(authors_list) != 0:
        for author in  authors_list:
            #it is a list of genres
            authorsID_list.append(author['author_id'])
    return authorsID_list
def get_authorName(author_data):
    author_dict=json.loads(author_data)
    authorsID_names_list=[]
    id=author_dict['author_id']
    name=author_dict['name']
    authorsID_names_list.append(id+'***'+name)
    return authorsID_names_list
    #return (id,name)


from itertools import product
def get_writerID_with_genre(book_metadata):
    """
    this function output every genre assigned to authors of every book in the book_metadata goodreads_books.json.gz
    """
    writer_genre_list=[]
    #avoid_list=['to-read','owned','currently-reading','favorites','books-i-own','kindle','ebook','library','default','owned-books','to-buy','ebooks','wish-list','my-books','my-library','e-book','audiobook','books','i-own','audiobooks','audio','read-in-2016','favourites','e-books','read-in-2015','read-in-2017','own-it','maybe','read-in-2014','abandoned','did-not-finish'] # add others if found
    lookup_list=['fiction','romance','adult','non-fiction','fantasy','mystery','novels','nonfiction','history','historical','young-adult','historical-fiction','adult-fiction','adventure','literature','thriller','classics','suspense','paranormal','science-fiction','crime','drama','sci-fi','humor','contemporary-romance','general-fiction','contemporary-fiction','family','school','mystery-thriller','childrens','short-stories','children','biography','classic','literary-fiction','sci-fi-fantasy','horror','supernatural','mysteries','magic','american','children-s','action','kids','mystery-suspense','british','netgalley','science','children-s-books','stand-alone','philosophy','erotica','teen','urban-fantasy','funny','religion','fantasy-sci-fi','realistic-fiction','literary','scifi','politics','memoir','love','war','childrens-books','paranormal-romance','thrillers','humour','europe','friendship','childhood','scifi-fantasy','comics','psychology','short-story','kids-books','women','detective','graphic-novels','action-adventure','crime-fiction','middle-grade']
    genre_list=[]
    authors_list=[]
    book_dict=json.loads(book_metadata)
    if len(book_dict['popular_shelves']) != 0:
        for genre in  book_dict['popular_shelves']:
            #it is a list of genres
            if genre['name'] in lookup_list:
                genre_list.append(genre['name'])
    if len(book_dict['authors'])!=0 and len(genre_list)!=0:
        for author in book_dict['authors']:
            authors_list.append(author['author_id'])
        writer_genre_list=list(product(authors_list,genre_list))
        writer_genre_list=[i[0]+'***'+i[1] for i in writer_genre_list]

    return writer_genre_list

pathw=''
#myfunction= get_2genre_combination
fileName='/content/drive/MyDrive/BigData_FinalProject/Output/Top_Author_with_genre/top_author_genre.txt'

In [ ]:
words=books_data.flatMap(lambda line:myfunction(line))
wordcount=words.map(lambda key:(key,1)).reduceByKey(lambda key,value: key+value).sortBy(lambda x: x[1], ascending=False)
result = wordcount.collect()

In [ ]:
words=books_data.flatMap(lambda line:get_writerID_with_genre(line))
data1=words.map(lambda key:(key.split('***')[0],key.split('***')[1])) #(id,genre)  (id,name)>> (id,(name,genre))>> ((name,genre),number)
#data1_mod=data1.map(lambda line: (line[0].split('***')[0],line[0].split('***')[1]+'***'+line[1]))
data2=author_data.flatMap(lambda line:get_authorName(line)).map(lambda line: (line.split('***')[0],line.split('***')[1]))
out=data2.join(data1)
out2=out.map(lambda key:(key[1],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False)
result = out2.collect()

In [ ]:
result

[(('Agatha Christie', 'mystery'), 4307),
 (('Stephen King', 'horror'), 4248),
 (('Stephen King', 'fiction'), 4173),
 (('Agatha Christie', 'fiction'), 4170),
 (('Agatha Christie', 'crime'), 4106),
 (('Agatha Christie', 'classics'), 3998),
 (('Agatha Christie', 'mysteries'), 3995),
 (('Agatha Christie', 'mystery-thriller'), 3957),
 (('Agatha Christie', 'british'), 3950),
 (('Agatha Christie', 'classic'), 3943),
 (('Agatha Christie', 'mystery-suspense'), 3905),
 (('Agatha Christie', 'detective'), 3890),
 (('Agatha Christie', 'thriller'), 3878),
 (('Stephen King', 'fantasy'), 3855),
 (('Agatha Christie', 'crime-fiction'), 3819),
 (('Agatha Christie', 'suspense'), 3817),
 (('Agatha Christie', 'novels'), 3794),
 (('Agatha Christie', 'adult'), 3718),
 (('Stephen King', 'adult'), 3694),
 (('Agatha Christie', 'adult-fiction'), 3681),
 (('Stephen King', 'thriller'), 3618),
 (('Stephen King', 'mystery'), 3583),
 (('Stephen King', 'suspense'), 3478),
 (('Stephen King', 'american'), 3400),
 (('Step

In [ ]:
with open(fileName,'w+') as file:
  file.write(
      '\n'.join(str(i) for i in result)
  )